In [4]:
!pip install webdriver_manager

In [7]:
!pip install selenium==4.2.0

  Attempting uninstall: selenium
    Found existing installation: selenium 4.4.0
    Uninstalling selenium-4.4.0:
      Successfully uninstalled selenium-4.4.0


In [24]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

email = ""
password = ""
driver.get("https://www.linkedin.com/login")
element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "username")))

email_elem = driver.find_element_by_id("username")
email_elem.send_keys(email)

password_elem = driver.find_element_by_id("password")
password_elem.send_keys(password)
password_elem.submit()

driver.implicitly_wait(3)

C:\Users\yong2\AppData\Local\Temp\ipykernel_28248\304109516.py:18: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  email_elem = driver.find_element_by_id("username")
C:\Users\yong2\AppData\Local\Temp\ipykernel_28248\304109516.py:21: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  password_elem = driver.find_element_by_id("password")


In [2]:
pages = range(25)
links = []
for page in pages:
    url_search_by_people = f"https://www.linkedin.com/search/results/people/?keywords=Data%20Scientist&origin=SWITCH_SEARCH_VERTICAL&page={page+15}"
    driver.get(url_search_by_people)
    elems = driver.find_elements_by_css_selector(".mb1 a.app-aware-link")
    for elem in elems:
        link = elem.get_attribute('href').split("?")[0]
        if link != 'https://www.linkedin.com/search/results/people/headless':
            links.append(link)


C:\Users\yong2\AppData\Local\Temp\ipykernel_28248\1614278513.py:6: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  elems = driver.find_elements_by_css_selector(".mb1 a.app-aware-link")


In [56]:
links = ['https://www.linkedin.com/in/pooya-movahed-b897b981', 'https://www.linkedin.com/in/emmayaoxie', 'https://www.linkedin.com/in/xiaozhou-shen', 'https://www.linkedin.com/in/yacine-brini', 'https://www.linkedin.com/in/eddy-yeo-wai-kit', 'https://www.linkedin.com/in/yixuanliu717', 'https://www.linkedin.com/in/rahul-raman-0958209b', 'https://www.linkedin.com/in/tycatherine-jiang', 'https://www.linkedin.com/in/ravi-yadav', 'https://www.linkedin.com/in/gabriele-boncoraglio', 'https://www.linkedin.com/in/ikespand', 'https://www.linkedin.com/in/missmalvika', 'https://www.linkedin.com/in/shravan-sarma-devarakonda-4616445b', 'https://www.linkedin.com/in/alvinchiew', 'https://www.linkedin.com/in/yingtingloo', 'https://www.linkedin.com/in/fadhlirosli', 'https://www.linkedin.com/in/patilvc', 'https://www.linkedin.com/in/yeow-tk', 'https://www.linkedin.com/in/sunil-kafle', 'https://www.linkedin.com/in/parisgu', 'https://www.linkedin.com/in/nate-ehat', 'https://www.linkedin.com/in/benjamin-amaral', 'https://www.linkedin.com/in/faysalabdulaziz', 'https://www.linkedin.com/in/hari-kumar-n-3a4064119', 'https://www.linkedin.com/in/cicicemal', 'https://www.linkedin.com/in/denistanjingyu', 'https://www.linkedin.com/in/ahmad-alif-imran-77bb1a119', 'https://www.linkedin.com/in/azizarrashid', 'https://www.linkedin.com/in/woon-yen-pang-bba36815b', 'https://www.linkedin.com/in/ethan-jeremiah-chitty-88564416a', 'https://www.linkedin.com/in/dandanfeng', 'https://www.linkedin.com/in/yana-cheng-49969514', 'https://www.linkedin.com/in/choon-yan-teh-6852a015', 'https://www.linkedin.com/in/cheang-yee-mun-0b463a117', 'https://www.linkedin.com/in/sijingtoh', 'https://www.linkedin.com/in/sze-hui-wong-a55311118', 'https://www.linkedin.com/in/y-k-yuan-kai-tan-648b41129', 'https://www.linkedin.com/in/amir-aminuddin-azland-332385166', 'https://www.linkedin.com/in/nuur-fahima-husna-ghazali-21ba76a3', 'https://www.linkedin.com/in/weixinooi', 'https://www.linkedin.com/in/mjthou', 'https://www.linkedin.com/in/alvintanjianjia', 'https://www.linkedin.com/in/ashleythy', 'https://www.linkedin.com/in/abhishek-mahindrakar-csm%C2%AE-88b3641b', 'https://www.linkedin.com/in/izzudinhussein', 'https://www.linkedin.com/in/shafiqalias', 'https://www.linkedin.com/in/weihanler', 'https://www.linkedin.com/in/shareh-zulhelmi-496899159', 'https://www.linkedin.com/in/nicolenlchong', 'https://www.linkedin.com/in/wong-rui-yun-a1915a121', 'https://www.linkedin.com/in/ginnylu', 'https://www.linkedin.com/in/arif-amin-ahmad-puzi', 'https://www.linkedin.com/in/limyixiang', 'https://www.linkedin.com/in/ji-dian-yeoh-374ab855', 'https://www.linkedin.com/in/mohd-khairul-imran-a-kadir-46721264', 'https://www.linkedin.com/in/hasmirah-hassan', 'https://www.linkedin.com/in/farhankamaludin', 'https://www.linkedin.com/in/joshcx', 'https://www.linkedin.com/in/ernest-chiew', 'https://www.linkedin.com/in/osama-ali-ahmed', 'https://www.linkedin.com/in/kenyap95', 'https://www.linkedin.com/in/parvin-singh', 'https://www.linkedin.com/in/chi-jiang-edwin-loh-319209127', 'https://www.linkedin.com/in/elkhan-gahramanli', 'https://www.linkedin.com/in/fzdanial', 'https://www.linkedin.com/in/zohreh-madhoushi-a710b199', 'https://www.linkedin.com/in/ali-aryanfar', 'https://www.linkedin.com/in/daniel-samwel-amon-ochieng-otieno-ba28a813', 'https://www.linkedin.com/in/aqilahusnaamdan', 'https://www.linkedin.com/in/nfarahsyazwani', 'https://www.linkedin.com/in/kelvin-ng-han-yao-abb7a5145', 'https://www.linkedin.com/in/teo-yong-zheng-712183135', 'https://www.linkedin.com/in/alyariad', 'https://www.linkedin.com/in/muhammadfairuz', 'https://www.linkedin.com/in/mohamad-syafiq-75a11b108', 'https://www.linkedin.com/in/waijianlim', 'https://www.linkedin.com/in/scholarazwa', 'https://www.linkedin.com/in/teshwinagilan', 'https://www.linkedin.com/in/albert-pee-ka-zhen-9b343218b', 'https://www.linkedin.com/in/leong-siew-hua', 'https://www.linkedin.com/in/jiayonggan', 'https://www.linkedin.com/in/farha-rahman-644591117', 'https://www.linkedin.com/in/mei-pheng-tan-70a602130', 'https://www.linkedin.com/in/vicky-ali-48774129', 'https://www.linkedin.com/in/syahnizal', 'https://www.linkedin.com/in/seng-yui-tan-901530196', 'https://www.linkedin.com/in/haniff-hamdan-138908201', 'https://www.linkedin.com/in/sheh-jing-tan-807631107', 'https://www.linkedin.com/in/edmundhongkf', 'https://www.linkedin.com/in/muhammad-nurhakim-mohamed-bazet-917a65101', 'https://www.linkedin.com/in/nur-arisha-mohd-azhar', 'https://www.linkedin.com/in/azzubair-azeman-b96222142', 'https://www.linkedin.com/in/sook-fei-chia-b969a4160', 'https://www.linkedin.com/in/slyvia-tan', 'https://www.linkedin.com/in/taofiqbakarebeng', 'https://www.linkedin.com/in/izzahkhumaira', 'https://www.linkedin.com/in/ivan-gan-hong-oon', 'https://www.linkedin.com/in/max-yap', 'https://www.linkedin.com/in/gs-ts-dr-fauzy-che-yayah-20628426', 'https://www.linkedin.com/in/syaza-subhi-b81467147', 'https://www.linkedin.com/in/seemun-yum', 'https://www.linkedin.com/in/khairulnizam', 'https://www.linkedin.com/in/nikin-m-3a088a117', 'https://www.linkedin.com/in/ming-yoong-89238a183', 'https://www.linkedin.com/in/johncys', 'https://www.linkedin.com/in/glennchiajunweng', 'https://www.linkedin.com/in/bengsoon', 'https://www.linkedin.com/in/terence-toh-zhen-jian-10a922195', 'https://www.linkedin.com/in/kamarulhisham', 'https://www.linkedin.com/in/kehui-tee', 'https://www.linkedin.com/in/nabilaljohori', 'https://www.linkedin.com/in/rakesh-bahwandi-a28b15101', 'https://www.linkedin.com/in/wei-sheng-tan-597270187', 'https://www.linkedin.com/in/idzwein', 'https://www.linkedin.com/in/geraldin-divya-246236105', 'https://www.linkedin.com/in/cheang-hoi-him', 'https://www.linkedin.com/in/nur-nesa-nashuha-ismail-4368a710b', 'https://www.linkedin.com/in/hazirahindra', 'https://www.linkedin.com/in/kokbing96', 'https://www.linkedin.com/in/nuralyaanadhirah', 'https://www.linkedin.com/in/qurratu-aini-binti-musa-1a1941170', 'https://www.linkedin.com/in/fadzlifuzi', 'https://www.linkedin.com/in/andrew-rashid-day-53941214b', 'https://www.linkedin.com/in/narjes-khatoon-naseri-4590bbb8', 'https://www.linkedin.com/in/yasmin-zulaikha-84b05b206', 'https://www.linkedin.com/in/khoonkheng', 'https://www.linkedin.com/in/muhammedhaziq', 'https://www.linkedin.com/in/kaichyuan', 'https://www.linkedin.com/in/naimiskandar22', 'https://www.linkedin.com/in/sai-weng-ng-807740207', 'https://www.linkedin.com/in/lpshyuan', 'https://www.linkedin.com/in/sitisarahahmadsuhaimi', 'https://www.linkedin.com/in/cevest-y-a328985a', 'https://www.linkedin.com/in/yaqim-othman-74874a138', 'https://www.linkedin.com/in/sarah-hazwani-abdul-rahman-b0abb9108', 'https://www.linkedin.com/in/kian-huat-ng-58263678', 'https://www.linkedin.com/in/firdaus-khalid', 'https://www.linkedin.com/in/shaffilza', 'https://www.linkedin.com/in/paulo-dias-cia-827604', 'https://www.linkedin.com/in/amieruddin-zuki', 'https://www.linkedin.com/in/rebekahyg', 'https://www.linkedin.com/in/wei-lin-chin-6738b7171', 'https://www.linkedin.com/in/mashkur-billah-7b22441aa', 'https://www.linkedin.com/in/vigneshsubramanian25', 'https://www.linkedin.com/in/stephen-adedigba-990520227', 'https://www.linkedin.com/in/kong-jun-ng-375802225', 'https://www.linkedin.com/in/lai-kua-wa-ab4415216', 'https://www.linkedin.com/in/leong-khai-herng-2152a6212', 'https://www.linkedin.com/in/omar-sazali-68a235230', 'https://www.linkedin.com/in/yikherr', 'https://www.linkedin.com/in/azeemshah-amiruddin', 'https://www.linkedin.com/in/nachai-limsettho-ph-d-7609b8120', 'https://www.linkedin.com/in/vivek6449', 'https://www.linkedin.com/in/pierre-cayot-400734a4', 'https://www.linkedin.com/in/sriamirthan', 'https://www.linkedin.com/in/ofirmagdaci', 'https://www.linkedin.com/in/jannikwiessler', 'https://www.linkedin.com/in/evren-bayraktar-725151152', 'https://www.linkedin.com/in/ming-hao-teo-40b12256', 'https://www.linkedin.com/in/leilei-zhang-32728583', 'https://www.linkedin.com/in/elton-savio-a309a258', 'https://www.linkedin.com/in/tianyi-tim-zhang-38608435', 'https://www.linkedin.com/in/xiangli2533', 'https://www.linkedin.com/in/guangtingzhou', 'https://www.linkedin.com/in/williamardianto', 'https://www.linkedin.com/in/bayan-saparbayeva', 'https://www.linkedin.com/in/yi-lu-nyu', 'https://www.linkedin.com/in/krishnendu-chakraborty-iitd', 'https://www.linkedin.com/in/pranay-kale-11ba6a30', 'https://www.linkedin.com/in/han-zhu-derekchu', 'https://www.linkedin.com/in/shan-sung-liew', 'https://www.linkedin.com/in/siti-nadiyah-kader-0448076b', 'https://www.linkedin.com/in/jingwei-too-1b62b41b0', 'https://www.linkedin.com/in/ankethm', 'https://www.linkedin.com/in/humairah-yahya-80496a1b2', 'https://www.linkedin.com/in/muaz-rahim', 'https://www.linkedin.com/in/ritesh-mohan-srivastava-15852465', 'https://www.linkedin.com/in/nuryusra-arafah-bt-md-isa-yusuff-b0190a207', 'https://www.linkedin.com/in/ahmed-yahia-trad-7bb1b793', 'https://www.linkedin.com/in/divyanshu-vyas', 'https://www.linkedin.com/in/tbergasse', 'https://www.linkedin.com/in/bruno-chagas', 'https://www.linkedin.com/in/vamshi-gorantla', 'https://www.linkedin.com/in/sachinsanand', 'https://www.linkedin.com/in/alexandre-felix-16288a34', 'https://www.linkedin.com/in/xiaoguai-li-2b88236b', 'https://www.linkedin.com/in/mohd-faidzal-mat-lazin-10b731128', 'https://www.linkedin.com/in/jie-yuan-frm-caia-03501a127', 'https://www.linkedin.com/in/yangfan255', 'https://www.linkedin.com/in/alex-wei-zeng-low-phd-4309aa142', 'https://www.linkedin.com/in/xun-guo-w-5b5b81155', 'https://www.linkedin.com/in/raj-saraiya-408061103', 'https://www.linkedin.com/in/shuhaili-safee-45500678', 'https://www.linkedin.com/in/syed-faris-akmal-syed-zulkhieree', 'https://www.linkedin.com/in/khairiraimi', 'https://www.linkedin.com/in/naelahnordin', 'https://www.linkedin.com/in/kharshin', 'https://www.linkedin.com/in/charles-cmu', 'https://www.linkedin.com/in/ainnur-farah-b49a7b191', 'https://www.linkedin.com/in/natashabuhary', 'https://www.linkedin.com/in/zulfiaftar-abd-jalil-878598174', 'https://www.linkedin.com/in/eilyn-chong', 'https://www.linkedin.com/in/fakhrulammar', 'https://www.linkedin.com/in/wan-wahida-91b368209', 'https://www.linkedin.com/in/nikamalaiman', 'https://www.linkedin.com/in/xuyuhe', 'https://www.linkedin.com/in/irfan-kadikoylu-45b761bb', 'https://www.linkedin.com/in/sepideh-mesbah-108b4121', 'https://www.linkedin.com/in/junting-chen-20ab6137', 'https://www.linkedin.com/in/fatin-nadiah-mohd-razib-081609225', 'https://www.linkedin.com/in/nkwaffa', 'https://www.linkedin.com/in/nurzawanah-zuhairah-014558213', 'https://www.linkedin.com/in/kong-you-liow-33078422b', 'https://www.linkedin.com/in/basem-abdelwahab-117323233', 'https://www.linkedin.com/in/yik-siang-pang-55142920b', 'https://www.linkedin.com/in/nurzahirah-mohd-muhiden-5b30851a8', 'https://www.linkedin.com/in/iylia-ezaty', 'https://www.linkedin.com/in/muhammad-hadi-mohd-zaki-b9407115b', 'https://www.linkedin.com/in/nurul-ashikin-samat-b23ba7134', 'https://www.linkedin.com/in/lee-yong-xian-47968a7b', 'https://www.linkedin.com/in/miguel-prytoluk', 'https://www.linkedin.com/in/zaraazam18', 'https://www.linkedin.com/in/noor-adilah-rashid', 'https://www.linkedin.com/in/roobharajamohan', 'https://www.linkedin.com/in/nrmzmh', 'https://www.linkedin.com/in/winsonyeap', 'https://www.linkedin.com/in/qabd88', 'https://www.linkedin.com/in/razira-aniza-roslan-991b40145', 'https://www.linkedin.com/in/dr-nur-ayuni-mohamed-2896218a', 'https://www.linkedin.com/in/zhi-yang-ooi-170735181', 'https://www.linkedin.com/in/wbtan7', 'https://www.linkedin.com/in/shenghan', 'https://www.linkedin.com/in/han-ming-cheah-5166b6127', 'https://www.linkedin.com/in/rusernyeoh123', 'https://www.linkedin.com/in/ooihs', 'https://www.linkedin.com/in/peirkern', 'https://www.linkedin.com/in/khai-ping-lai-65bb0a193', 'https://www.linkedin.com/in/yi-wei-tye-689072132', 'https://www.linkedin.com/in/chee-siang-tan-0b9223144', 'https://www.linkedin.com/in/weisheng4321', 'https://www.linkedin.com/in/puteri-hasya-damia', 'https://www.linkedin.com/in/stephen-kua-032959140', 'https://www.linkedin.com/in/chongjeeseng', 'https://www.linkedin.com/in/chinwaizhen', 'https://www.linkedin.com/in/ericlowkimhoe', 'https://www.linkedin.com/in/steven-tan-chee-yu-790143136', 'https://www.linkedin.com/in/a-luqman-hakim-mohd-yunus', 'https://www.linkedin.com/in/teo-boon-long-487712116', 'https://www.linkedin.com/in/vincenttoong', 'https://www.linkedin.com/in/jia-wei-teh', 'https://www.linkedin.com/in/chee-yau-kee-17293087', 'https://www.linkedin.com/in/yewjie', 'https://www.linkedin.com/in/wayne-t-w-ng-80275637', 'https://www.linkedin.com/in/boon-khai-yeoh', 'https://www.linkedin.com/in/siewchen', 'https://www.linkedin.com/in/darveenvijayan', 'https://www.linkedin.com/in/chun-wen-khoo', 'https://www.linkedin.com/in/shangcheongwong', 'https://www.linkedin.com/in/dr-pranesh-krishnan', 'https://www.linkedin.com/in/ching-herny-chin', 'https://www.linkedin.com/in/divya-dhandevi-woodun-18b467ba', 'https://www.linkedin.com/in/mohamed-lubani', 'https://www.linkedin.com/in/kianchongkhoo', 'https://www.linkedin.com/in/chuahanchong', 'https://www.linkedin.com/in/kamal-izzudin-idris-6194741b1', 'https://www.linkedin.com/in/kohguanli', 'https://www.linkedin.com/in/amerwafiy', 'https://www.linkedin.com/in/kawserahmedpinto', 'https://www.linkedin.com/in/asadullah-qamar-bhatti', 'https://www.linkedin.com/in/wernyitan', 'https://www.linkedin.com/in/huan-yang-chan-a594a7142', 'https://www.linkedin.com/in/brendon-soong-b9128a181', 'https://www.linkedin.com/in/choy-siew-wearn-970095ab', 'https://www.linkedin.com/in/john-syin-ang', 'https://www.linkedin.com/in/ravindu-thaveesha', 'https://www.linkedin.com/in/jayakumar-viswanathan-48b53b147', 'https://www.linkedin.com/in/chingwaiyong', 'https://www.linkedin.com/in/prakash-gyan', 'https://www.linkedin.com/in/ee-kein-ivan-boo-b300941a0', 'https://www.linkedin.com/in/yiwei-ang-412bba138', 'https://www.linkedin.com/in/ammar-azman', 'https://www.linkedin.com/in/jai-wei-choo', 'https://www.linkedin.com/in/khairmiji', 'https://www.linkedin.com/in/jovinytliew', 'https://www.linkedin.com/in/sulaihasubi', 'https://www.linkedin.com/in/weng-hoong-seng', 'https://www.linkedin.com/in/marcusnaik', 'https://www.linkedin.com/in/amzar-muzani-bin-maarof', 'https://www.linkedin.com/in/yi-qin-teow-b847981b1', 'https://www.linkedin.com/in/hester-lim-569a27ab', 'https://www.linkedin.com/in/aarontanx', 'https://www.linkedin.com/in/ming-xiang-lee-23610a95', 'https://www.linkedin.com/in/justinloke', 'https://www.linkedin.com/in/thangeswariy', 'https://www.linkedin.com/in/ezonneoh', 'https://www.linkedin.com/in/gv-r-56780120a', 'https://www.linkedin.com/in/tianyan', 'https://www.linkedin.com/in/yew-leong-toh-04210975', 'https://www.linkedin.com/in/faiz-jim', 'https://www.linkedin.com/in/thongweiseng', 'https://www.linkedin.com/in/leezing', 'https://www.linkedin.com/in/sheikhhanif', 'https://www.linkedin.com/in/saibal13', 'https://www.linkedin.com/in/ngjiawaie', 'https://www.linkedin.com/in/mypratik', 'https://www.linkedin.com/in/georgewongsinhong', 'https://www.linkedin.com/in/ahmadzuhair', 'https://www.linkedin.com/in/jia-chun-tan-8b73051a0', 'https://www.linkedin.com/in/tanpeiseng', 'https://www.linkedin.com/in/kesava-a-88272789', 'https://www.linkedin.com/in/seantanrh', 'https://www.linkedin.com/in/william-yap', 'https://www.linkedin.com/in/ann-nee-wong', 'https://www.linkedin.com/in/tan-kai-4975a11a9', 'https://www.linkedin.com/in/santosh-kumar-79a447b2', 'https://www.linkedin.com/in/kishor-magar-57a98023', 'https://www.linkedin.com/in/chanzhenyu', 'https://www.linkedin.com/in/muhammad-aqhari-nasrin-bin-ramli', 'https://www.linkedin.com/in/khawslee', 'https://www.linkedin.com/in/limzhongheng', 'https://www.linkedin.com/in/sitisarahzab', 'https://www.linkedin.com/in/cody-c-a37b8918a', 'https://www.linkedin.com/in/fatin-amani-mohd-ali-87b614190', 'https://www.linkedin.com/in/faris-omar-b27a56144', 'https://www.linkedin.com/in/sze-yann-tham-796515190', 'https://www.linkedin.com/in/muhammad-anas-husnan-55a482b1', 'https://www.linkedin.com/in/jasbir-singh-32a83920', 'https://www.linkedin.com/in/li-vern-tan', 'https://www.linkedin.com/in/yx-00168', 'https://www.linkedin.com/in/kexin-lee-222374122', 'https://www.linkedin.com/in/daniel-yusoff-a40a4b166', 'https://www.linkedin.com/in/thkhaw', 'https://www.linkedin.com/in/ngkeanchew', 'https://www.linkedin.com/in/meng-kuan-lee-1089', 'https://www.linkedin.com/in/chiqin-lai-phd-4568a2119', 'https://www.linkedin.com/in/isaac-tan-how-ann-98147918a', 'https://www.linkedin.com/in/jing-yi-ong-mango1105', 'https://www.linkedin.com/in/vishal-singh-27a37838', 'https://www.linkedin.com/in/hariharan-muniandy-88a84817a', 'https://www.linkedin.com/in/muhamad-haris-idris-463bb6114', 'https://www.linkedin.com/in/philip-khor', 'https://www.linkedin.com/in/hui-theng-chew-leong-17119a193', 'https://www.linkedin.com/in/chungmeng', 'https://www.linkedin.com/in/thevesh', 'https://www.linkedin.com/in/abubakar-bala', 'https://www.linkedin.com/in/m%C3%A5rten-bj%C3%B6rketun-12505536', 'https://www.linkedin.com/in/bh-tan', 'https://www.linkedin.com/in/aqilahrose']
print(len(links))

348


In [61]:
from selenium.common.exceptions import NoSuchElementException
import time
import pandas as pd 
from selenium.webdriver.common.by import By

#i try avoid list comprehension here, fearing it adds overhead
candidates = []
driver.implicitly_wait(2)
count = 0
for link in links:
    count = count + 1
    if count <= 130:
        continue
    if count > 150:
        break
    print(f"processing {link}")
    driver.get(link)
    
    elem_name = driver.find_element(By.CLASS_NAME, "text-heading-xlarge").text
    elem_title = driver.find_element(By.CSS_SELECTOR, "div.pv-text-details__left-panel > div.text-body-medium").text
    elem_loc = driver.find_element(By.CSS_SELECTOR, "div.pv-text-details__left-panel > span.text-body-small").text
    
    es_ = driver.find_elements(By.CSS_SELECTOR, "section.artdeco-card.ember-view.relative.break-words")
    
    time.sleep(1)    
    ###Experience###
    section_exp = []
    for e_ in es_:
        div_exps = e_.find_elements(By.CSS_SELECTOR, "div#experience")
        if len(div_exps) > 0:
            section_exp.append(e_)
            break
    
    if len(section_exp) > 0:
        elem_exp = section_exp[0]
        elems_exp = elem_exp.find_elements(By.CSS_SELECTOR, "div.pvs-list__outer-container div.display-flex.flex-column.full-width.align-self-center")

        exps = []
        for elem_exp in elems_exp:
            spans = elem_exp.find_elements(By.CSS_SELECTOR, "span.t-14 > span")
            exp = [span.text for span in spans if span.get_attribute("aria-hidden") is not None]
            
            title_spans = elem_exp.find_elements(By.CSS_SELECTOR, "span.mr1 > span")
            titles = [span.text for span in title_spans if span.get_attribute("aria-hidden") is not None]
            exps.append({"titles":titles, "exp": exp})
    ##################
    
    ###Education###
    section_edu = []
    for e_ in es_:
        div_edus = e_.find_elements(By.CSS_SELECTOR, "div#education")
        if len(div_edus) > 0:
            section_edu.append(e_)
            break
    
    if len(section_edu) > 0:
        elem_edu = section_edu[0]
        elems_edu = elem_edu.find_elements(By.CSS_SELECTOR, "div.pvs-list__outer-container div.display-flex.flex-column.full-width.align-self-center")

        edus = []
        for elem_edu in elems_edu:
            spans = elem_edu.find_elements(By.CSS_SELECTOR, "span.t-14 > span")
            edu = [span.text for span in spans if span.get_attribute("aria-hidden") is not None]
            edus.append(edu)
            
            
    ###Skills###
    skill_page_url = f"{link}/details/skills"
    print(skill_page_url)
    driver.get(skill_page_url)
    time.sleep(2)    
   
    tt = driver.find_elements(By.CSS_SELECTOR, "section.artdeco-card.ember-view")
    if len(tt) > 0:
        span_skills = tt[0].find_elements(By.CSS_SELECTOR, "div.display-flex.align-items-center span.mr1 > span")
        skills = list(set([span.text for span in span_skills if span.get_attribute("aria-hidden") is not None]))
        skills = [skill for skill in skills if skill]
        
    candidate = {"link": link, "name": elem_name, "current_title": elem_title, "current_location": elem_loc, 
                           "exps": exps, "edus": edus, "skills": skills}
    
    df = pd.DataFrame([candidate])
    df.to_csv('full-all-2.csv', encoding="utf-16", header=False, mode='a', index=False)
    
    candidates.append(candidate)
    
    
print(f"Done processing {len(candidates)} candidates")

processing https://www.linkedin.com/in/seemun-yum
https://www.linkedin.com/in/seemun-yum/details/skills
processing https://www.linkedin.com/in/khairulnizam
https://www.linkedin.com/in/khairulnizam/details/skills
processing https://www.linkedin.com/in/nikin-m-3a088a117
https://www.linkedin.com/in/nikin-m-3a088a117/details/skills
processing https://www.linkedin.com/in/ming-yoong-89238a183
https://www.linkedin.com/in/ming-yoong-89238a183/details/skills
processing https://www.linkedin.com/in/johncys
https://www.linkedin.com/in/johncys/details/skills
processing https://www.linkedin.com/in/glennchiajunweng
https://www.linkedin.com/in/glennchiajunweng/details/skills
processing https://www.linkedin.com/in/bengsoon
https://www.linkedin.com/in/bengsoon/details/skills
processing https://www.linkedin.com/in/terence-toh-zhen-jian-10a922195
https://www.linkedin.com/in/terence-toh-zhen-jian-10a922195/details/skills
processing https://www.linkedin.com/in/kamarulhisham
https://www.linkedin.com/in/kamar

In [ ]:
#Debugging if necessary
print(candidates)

###### IGNORE CELLS BELOW#############

#person_url = "https://www.linkedin.com/in/yong-yong-khoo-6597a3137/"
#person_url2 = "https://www.linkedin.com/in/joey-sham-aa2a50122/"

#s_input = driver.find_element_by_css_selector("#global-nav-typeahead > input")
#s_input.send_keys("Data Scientist")
#s_input.send_keys(Keys.ENTER)

#driver.get(links[2])
#driver.implicitly_wait(10)

#elems_with_artdeco = driver.find_elements_by_css_selector("section.artdeco-card.ember-view.relative.break-words")
#print(len(elems))
#print(elem_by_xpath)
#print(elem_by_xpath.get_attribute("innerHTML"))
#print(elems_exp.get_attribute("innerHTML"))
#div.display-flex.flex-column.full-width

In [ ]:
    """   
    #Section 4 = Experience
    time.sleep(1)
    exp_xpath = "/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[4]"
    exp_xpath = "/html/body/div[4]/div[3]/div/div/div[2]/div/div/main/section[5]"
    #WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "experience")))
    
    elem_by_xpath = driver.find_element_by_xpath(exp_xpath)
    elems_exp = elem_by_xpath.find_elements_by_css_selector("div.pvs-list__outer-container div.display-flex.flex-column.full-width.align-self-center")
    exps = []
    for elem_exp in elems_exp:
        spans = elem_exp.find_elements_by_css_selector("span.t-14 > span")
        exp = [span.text for span in spans if span.get_attribute("aria-hidden") is not None]
        exps.append(exp)
     
    #Section 5 = Education
    elem_by_xpath = driver.find_element_by_xpath("/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[5]")
    elems_edus = elem_by_xpath.find_elements_by_css_selector("div.pvs-list__outer-container div.display-flex.flex-column.full-width.align-self-center")
    edus = []
    for elem_edu in elems_edus:
        spans = elem_exp.find_elements_by_css_selector("span.t-14 > span")
        edu = [span.text for span in spans if span.get_attribute("aria-hidden") is not None]
        edus.append(edu)
    
    
    """
     '''
    section_skill = []
    for e_ in es_:
        div_skills = e_.find_elements_by_css_selector("div#skills")
        if len(div_skills) > 0:
            section_skill.append(e_)
            break
    
    if len(section_skill) > 0:
        elem_skill = section_skill[0]
        elems_skills = elem_skill.find_elements_by_css_selector("div.pvs-list__outer-container div.display-flex.flex-column.full-width.align-self-center")

        skills = []
        for elem_skill in elems_skills:
            spans = elem_skill.find_elements_by_css_selector("span.mr1 > span")
            skill = [span.text for span in spans if span.get_attribute("aria-hidden") is not None]
            skills.append(skill)

    candidate = {"link": link, "name": elem_name, "current_title": elem_title, "current_location": elem_loc, 
                           "exps": exps, "edus": edus, "skills": skills}
    candidates.append(candidate)
    '''